<a href="https://colab.research.google.com/github/itzahs/CNN-RS/blob/main/AllBands_Weights_BigEarthNet_BatchSize64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


## Get the dataset

In [ ]:
! pip install -qU tensorflow_datasets

     |████████████████████████████████| 4.0 MB 9.5 MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks"


(train, val, test), ds_info = tfds.load("eurosat/all", 
                               with_info=True,
                               split=["train[:60%]", "train[60%:70%]", "train[70%:]"], #Train, val, test = 60, 10, 30
                               data_dir=DATA_DIR)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27000 [00:00<?, ? examples/s]

Shuffling eurosat-train.tfrecord...:   0%|          | 0/27000 [00:00<?, ? examples/s]

Dataset eurosat downloaded and prepared to /content/drive/MyDrive/Colab Notebooks/IGARSS/All/eurosat/all/2.0.0. Subsequent calls will reuse this data.


In [ ]:
#The data is in a dictionary with three keys: 'filename', 'image(as array)' and the 'label' 
train_point = next(iter(train)) #iterating over the dataset
print(tf.reduce_min(train_point["sentinel2"]))
print(tf.reduce_max(train_point["sentinel2"]))


tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(4812.0, shape=(), dtype=float32)


In [ ]:
test_point = next(iter(test)) #iterating over the dataset
print(tf.reduce_min(test_point["sentinel2"]))
print(tf.reduce_max(test_point["sentinel2"]))

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(5520.0, shape=(), dtype=float32)


In [ ]:
val_point = next(iter(val)) #iterating over the dataset
print(tf.reduce_min(val_point["sentinel2"]))
print(tf.reduce_max(val_point["sentinel2"]))

tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(5237.0, shape=(), dtype=float32)


## Parameters configuration and data preprocessing

In [ ]:
#Folder of the best model
FILE_MODEL = "'/content/drive/MyDrive/Colab Notebooks/Resnet50BigEarthNetAll64.hdf5'"

#Define main variables
NUM_EPOCHS = 100
BATCH_SIZE = 64
BUFFER_SIZE = 1000

IMAGE_SHAPE = [224, 224, 3]
NUM_CLASSES = ds_info.features["label"].num_classes

#As the dataset is batched is better to initialize:
STEPS_PER_EPOCH = int(ds_info.splits["train"].num_examples * 0.6)//BATCH_SIZE
VALIDATION_STEPS = int(ds_info.splits["train"].num_examples * 0.1)//BATCH_SIZE

In [ ]:
def data_preprocessing(datapoint):
  input_image = tf.image.resize(datapoint["sentinel2"], size=([IMAGE_SHAPE[0], IMAGE_SHAPE[1]])) #Size must be 1D-tensor of two elements: width and height
  
  input_image = input_image/65535           #Normalize uint16 values between 0 and 1
      
  return input_image, datapoint["label"]

In [ ]:
#Prepare data
train_dataset = train.map(data_preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE) #num_parallel_call is used for number of processors
validation_dataset = val.map(data_preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test.map(data_preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)

#Batch the data and use caching & prefetching to optimize loading speed
train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
#The data is in a dictionary with three keys: 'filename', 'image(as array)' and the 'label' 
train_norm = next(iter(train_dataset)) #iterating over the dataset
train_norm

(<tf.Tensor: shape=(64, 224, 224, 13), dtype=float32, numpy=
 array([[[[0.01908904, 0.0146334 , 0.01332113, ..., 0.02775616,
           0.01718166, 0.03495842],
          [0.01908904, 0.0146334 , 0.01332113, ..., 0.02775616,
           0.01718166, 0.03495842],
          [0.01908904, 0.0146334 , 0.01332113, ..., 0.02775616,
           0.01718166, 0.03495842],
          ...,
          [0.01941493, 0.01498545, 0.01251239, ..., 0.02397628,
           0.01540507, 0.03371917],
          [0.01942473, 0.01469444, 0.01173419, ..., 0.02386511,
           0.01527428, 0.03349355],
          [0.01942473, 0.01469444, 0.01173419, ..., 0.02386511,
           0.01527428, 0.03349355]],
 
         [[0.01908904, 0.0146334 , 0.01332113, ..., 0.02775616,
           0.01718166, 0.03495842],
          [0.01908904, 0.0146334 , 0.01332113, ..., 0.02775616,
           0.01718166, 0.03495842],
          [0.01908904, 0.0146334 , 0.01332113, ..., 0.02775616,
           0.01718166, 0.03495842],
          ...,
      

##Pre-trained models for image classification


### Load the model 


In [ ]:
# Load model
import tensorflow as tf             
import tensorflow_hub as hub
from keras.models import load_model

model_resnet = tf.keras.models.load_model((FILE_MODEL),custom_objects={'KerasLayer':hub.KerasLayer})
model_resnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 3)       42        
                                                                 
 keras_layer (KerasLayer)    (None, 2048)              23633899  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,688,159
Trainable params: 1,054,260
Non-trainable params: 23,633,899
_________________________________________________________________


##Evaluating the results 



In [ ]:
model_resnet.evaluate(test_dataset)

127/127 [==============================] - 15s 112ms/step - loss: 0.2915 - accuracy: 0.9406


[0.2915462851524353, 0.9406172633171082]